In [1]:
import jupyter_core
import os
from ipywidgets import *
from IPython.display import HTML, display, HTML
from notebook.services.config import ConfigManager
from __future__ import print_function

In [2]:
cdir = os.path.join(jupyter_core.paths.jupyter_config_dir(), 'custom')
ndir = os.path.join(jupyter_core.paths.jupyter_config_dir(), 'nbconfig')
css_filename = os.path.join(cdir, 'custom.css')
cjs_filename = os.path.join(cdir, 'custom.js')
edit_filename = os.path.join(ndir, 'edit.json')
cm = ConfigManager()
doc_tools_path = 'notebook-extensions-master/calysto/document-tools/main'

In [3]:
def create_form(desc, options, var, cb):
    label_layout = Layout(flex='1 1 auto')
    control_layout = Layout(flex='0 1 10%')
    form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        border='solid 1px lightgray',
        justify_content='space-between',
        width='60%'
    )
    w = Dropdown(options=options, value=var, layout=control_layout)
    w.observe(lambda x: cb(x['new']), names='value')
    return Box([Label(value=desc, layout=label_layout), w], layout=form_item_layout)

In [4]:
edit_config = """{
  "Editor": {
    "codemirror_options": {
      "vimMode": %s,
      "keyMap": "%s"
    }
  }
}
"""

editor_template = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/** KEYMAP %s **/
require(["codemirror/keymap/%s", "notebook/js/cell", "base/js/namespace"],
    function(emacs_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = '%s';
        var cells = IPython.notebook.get_cells();
        for(var c=0; c< cells.length ; c++){
            cells[c].code_mirror.setOption('keyMap', '%s');
            cells[cl].code_mirror.setOption("extraKeys", {
                "Ctrl-Enter": function(cm) {}
            });
        }
    }
);
"""

dash_css = """/****** DASHBOARD BUTTONS ******/
.jupyter-dashboard-toolbar-buttons {
    visibility: hidden;
}
"""

wide_css = """/* FULL WIDTH */
.notebook_app .container {
    width:99%; !important
}

/* Prevent the edit cell highlight box from getting clipped;
 * important so that it also works when cell is in edit mode*/
div.cell.selected {
    border-left-width: 1px !important;
}
"""

main_css = """/****** AUTOMATICALLY GENERATED BY NotebookSettings **************/

/* Warning: Anything you add here will get overwritten! */

/* remove clusters tab in tree view */
.clusters_tab_link {
    visibility: hidden;
}
/* remove CellToobar Reminder */
[title="show new celltoolbar selector location"] {
    display: none;
}
"""

eqn_js = """/** EQUATION NUMBERING **/
require([
    'base/js/namespace',
    'jquery'
],   function(IPython, $) {
    "use strict";

    IPython.toolbar.add_buttons_group([
        {
            id: 'reset_numbering',
            label: 'Reset equation numbering',
            icon: 'fa-sort-numeric-asc text-primary',
            callback: function () {
                MathJax.Hub.Queue(
                    ["resetEquationNumbers", MathJax.InputJax.TeX],
                    ["PreProcess", MathJax.Hub],
                    ["Reprocess", MathJax.Hub]
                );
                $('#reset_numbering').blur();
            }
        }
    ]);
    MathJax.Hub.Config({
        TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
});
"""

In [5]:
def update_css():
    global current_width, current_dash
    with open(css_filename, 'w') as f:
        f.write(main_css)
    if current_width == 'Wide':
        with open(css_filename, 'a') as f:
            f.write(wide_css)
    if current_dash == False:
        with open(css_filename, 'a') as f:
            f.write(dash_css)
    print("Please reload any open notebooks for the changes to take effect.")

def set_width(width):
    global current_width
    current_width = width
    update_css()

def set_editor(km):
    global current_eqn, current_editor
    current_editor = km

    # write edit.json
    with open(edit_filename, 'w') as f:
        f.write(edit_config % (str(km == 'vim').lower(), km))

    # write custom.js
    with open(cjs_filename, 'w') as f:
        f.write(editor_template % (km, km, km, km))

    # equation numbering
    if current_eqn:
        with open(cjs_filename, 'a') as f:
            f.write(eqn_js)
    print("Please reload any open notebooks for the changes to take effect.")

def set_eqn(new):
    global current_eqn, current_editor
    current_eqn = new
    set_editor(current_editor)
    print("Please reload any open notebooks for the changes to take effect.")

def set_doc_tools(new):
    global current_doc
    current_doc = new
    cm.update('notebook', {"load_extensions": {doc_tools_path: current_doc}})
    print("Please reload any open notebooks for the changes to take effect.")

def set_dash_tools(new):
    global current_dash
    current_dash = new
    update_css()

In [6]:
# create user config dirs if none exists
if not os.path.exists(cdir):
    os.system('mkdir -p ' + cdir)
if not os.path.exists(ndir):
    os.system('mkdir -p ' + ndir)

# read current values
current_editor = None
current_eqn = False
try:
    f = open(cjs_filename, 'r')
    for line in f.read().split('\n'):
        if line.startswith('/** KEYMAP '):
            current_editor = line.split()[2]
        elif line.startswith('/** EQUATION NUMBERING'):
            current_eqn = True
    f.close()
except:
    pass

if current_editor is None:
    current_editor = 'emacs'
    set_editor(current_editor)

current_width = None
current_dash = True
try:
    f = open(css_filename, 'r')
    for line in f.read().split('\n'):
        if line.startswith("/* FULL WIDTH */"):
            current_width = 'Wide'
        if line.startswith("/****** DASH"):
            current_dash = False
    f.close()
    if current_width is None:
        current_width = 'Normal'
except:
    pass
if current_width is None:
    current_width = 'Normal'
    set_width(current_width)


try:
    current_doc = cm.get('notebook')['load_extensions'][doc_tools_path]
except:
    cm.update('notebook', {"load_extensions": {doc_tools_path: False}})
    current_doc = False

# Notebook Settings

## Width
Sets the default cell width for the notebook.  Selecting 'wide' will cause the notebooks to fill the width of the browser window.

In [7]:
create_form("Notebook Width", ["Normal", "Wide"], current_width, set_width)

A Jupyter Widget

## Editor Mode
The notebook's built-in editor can emulate other popular editors.  

In [8]:
create_form("Keymap", {"Emacs": "emacs", "Vi": "vi", "Sublime": "sublime"}, current_editor, set_editor)

A Jupyter Widget

## Dashboard Tools

<img  src="dashboard.png" style="margin: 0"/>
Enables and disables the display of the dashboard tool buttons.  You will only need
these if you are creating dashboards (tools) instead of notebooks.

In [9]:
create_form("Dashboard Tools", {"OFF": False, "ON": True}, current_dash, set_dash_tools)

A Jupyter Widget

## Document Tools

<img  src="document_tools2.png" style="margin: 0"/>
If you want your notebook sections numbered and indexed, with references at the end, you should try this.  

Adds buttons to your menubar to move sections up and down, automatically number sections, generate a table of contents and generate references.

In [10]:
create_form("Document Tools", {"OFF": False, "ON": True}, current_doc, set_doc_tools)

A Jupyter Widget

## Equation Numbering

This will add an icon <img  src="renumber.png" style="margin: 0"/>
to the menubar which will renumber equations and generate references when clicked.
See [Equation Numbering](../manual/markdown.ipynb#Equation-Numbering-and-References) in the manual for more information.

In [11]:
create_form("Equation Numbering", {"OFF": False, "ON": True}, current_eqn, set_eqn)

A Jupyter Widget

In [12]:
input_form = """
<p>Clicking 'DONE' will exit this tool.  You should confirm if prompted by your browser.</p>
<hr>
<center><button id="stop_button" style="font-size: 24px;">DONE</button></center>
"""

js = """
<script type="text/Javascript">
$(document).ready(
function() {
    $("#stop_button").click(function() {
        var close_window = function () {
            /**
             * allow closing of new tabs in Chromium, impossible in FF
            */
            window.open('', '_self', '');
            window.close();
        };
        IPython.notebook.session.delete(close_window, close_window);
        });
    });
</script>
"""
display(HTML(input_form + js))